In [ ]:
import os
import json
import pandas as pd

In [176]:
import os
import json
import pandas as pd

folderPathPorData = '..\\..\\Bases\\PorData\\'

# Função para ler o arquivo JSON e retornar seu conteúdo como uma lista de dicionários
def read_json_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: O arquivo {file_path} não foi encontrado.")
    except json.JSONDecodeError:
        print(f"Error: Falha ao decodificar JSON no arquivo {file_path}.")
    return []

# Função para ler arquivos CSV a partir de uma pasta
def read_csv_files_from_folders(folder_path):
    dataframes = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)
                print(f'Processando arquivo: {file_path}')
                try:
                    df = pd.read_csv(file_path, sep=',', encoding='utf-8', low_memory=False)
                    dataframes.append({
                        'df_folder': os.path.basename(root),
                        'df_name': os.path.splitext(file)[0],
                        'df': df
                    })
                except Exception as e:
                    print(f"Erro ao ler {file}: {e}")
    return dataframes

# Função para tratar o DataFrame de cultura e cinemas
def trata_df_cultura(df_cultura_cinemas):
    df_cultura_cinemas = df_cultura_cinemas[df_cultura_cinemas['03. Âmbito Geográfico'].notnull()]
    last_year = df_cultura_cinemas['01. Ano'].max()
    df_cultura_cinemas = df_cultura_cinemas[df_cultura_cinemas['01. Ano'] == last_year]

    df_cultura_cinemas = df_cultura_cinemas[['01. Ano', '02. Nome Região (Portugal)', '03. Âmbito Geográfico', '09. Valor']]
    df_cultura_cinemas = df_cultura_cinemas.rename(columns={
        '01. Ano': 'Ano',
        '02. Nome Região (Portugal)': 'Região',
        '03. Âmbito Geográfico': 'Âmbito Geográfico',
        '09. Valor': 'Valor'
    })
    df_cultura_cinemas['Ano'] = df_cultura_cinemas['Ano'].astype(int)
    df_cultura_cinemas['Valor'] = df_cultura_cinemas['Valor'].astype(int)
    return df_cultura_cinemas.reset_index(drop=True)

# Função para encontrar um DataFrame pelo nome
def find_df_by_name(dataframes, df_name):
    for df_info in dataframes:
        if df_info['df_name'] == df_name:
            return df_info['df']
    raise ValueError(f"DataFrame com o nome '{df_name}' não encontrado.")

# Função para listar cidades com cinemas
def lista_cidades_cinemas(df_cultura_cinemas):
    df_cultura_cinemas = df_cultura_cinemas.groupby(['Região', 'Âmbito Geográfico']).agg({'Valor': 'sum'}).reset_index()
    df_cultura_cinemas = df_cultura_cinemas.sort_values(by='Valor', ascending=False).reset_index(drop=True)
    print("Cidades com mais cinemas:")
    for _, row in df_cultura_cinemas.iterrows():
        print(f"{row['Região']}: {row['Valor']} cinemas")

# Função para tratar o DataFrame de população residente
def trata_df_populacao_residente(df_populaccao_residente):
    df_populaccao_residente = df_populaccao_residente[df_populaccao_residente['04. Âmbito Geográfico'].notnull()]
    last_year = df_populaccao_residente['01. Ano'].max()
    df_populaccao_residente = df_populaccao_residente[df_populaccao_residente['01. Ano'] == last_year]

    df_sexo_pivot = df_populaccao_residente.pivot_table(
        index=['01. Ano', '03. Nome Região (Portugal)'],
        columns='05. Filtro 1',
        values='10. Valor',
        aggfunc='sum',
        fill_value=0
    )

    df_faixa_pivot = df_populaccao_residente.pivot_table(
        index=['01. Ano', '03. Nome Região (Portugal)'],
        columns='06. Filtro 2',
        values='10. Valor',
        aggfunc='sum',
        fill_value=0
    )

    df_polucao_final = pd.merge(df_sexo_pivot, df_faixa_pivot, on=['01. Ano', '03. Nome Região (Portugal)'], how='left')
    df_polucao_final.reset_index(inplace=True)
    df_polucao_final = df_polucao_final.rename(columns={
        '01. Ano': 'Ano',
        '03. Nome Região (Portugal)': 'Região',
        'Homens': 'Homens',
        'Mulheres': 'Mulheres',
        '0 - 4 anos': '0 - 4 anos',
        '5 - 9 anos': '5 - 9 anos',
        '10 - 14 anos': '10 - 14 anos',
        '15 - 19 anos': '15 - 19 anos',
        '20 - 24 anos': '20 - 24 anos',
        '25 - 29 anos': '25 - 29 anos',
        '30 - 34 anos': '30 - 34 anos',
        '35 - 39 anos': '35 - 39 anos',
        '40 - 44 anos': '40 - 44 anos',
        '45 - 49 anos': '45 - 49 anos',
        '50 - 54 anos': '50 - 54 anos',
        '55 - 59 anos': '55 - 59 anos',
        '60 - 64 anos': '60 - 64 anos',
        '65 - 69 anos': '65 - 69 anos',
        '70 - 74 anos': '70 - 74 anos',
        '75 - 79 anos': '75 - 79 anos',
        '80 - 84 anos': '80 - 84 anos',
        '85 ou mais anos': '85 ou mais anos',
        'Total_y': 'Total População'
    })

    df_polucao_final = df_polucao_final.astype({
        'Ano': 'int',
        'Região': 'str',
        'Homens': 'int',
        'Mulheres': 'int',
        '0 - 4 anos': 'int',
        '5 - 9 anos': 'int',
        '10 - 14 anos': 'int',
        '15 - 19 anos': 'int',
        '20 - 24 anos': 'int',
        '25 - 29 anos': 'int',
        '30 - 34 anos': 'int',
        '35 - 39 anos': 'int',
        '40 - 44 anos': 'int',
        '45 - 49 anos': 'int',
        '50 - 54 anos': 'int',
        '55 - 59 anos': 'int',
        '60 - 64 anos': 'int',
        '65 - 69 anos': 'int',
        '70 - 74 anos': 'int',
        '75 - 79 anos': 'int',
        '80 - 84 anos': 'int',
        '85 ou mais anos': 'int',
        'Total População': 'int'
    })

    # Calcular faixas etárias e proporções
    df_polucao_final['Total Crianças'] = df_polucao_final['0 - 4 anos'] + df_polucao_final['5 - 9 anos']
    df_polucao_final['Total Adolescentes'] = df_polucao_final['10 - 14 anos'] + df_polucao_final['15 - 19 anos']
    df_polucao_final['Total Jovens Adultos'] = df_polucao_final['20 - 24 anos']
    df_polucao_final['Total Adultos'] = df_polucao_final[['25 - 29 anos', '30 - 34 anos', '35 - 39 anos', '40 - 44 anos', 
                                                       '45 - 49 anos', '50 - 54 anos', '55 - 59 anos']].sum(axis=1)
    df_polucao_final['Total Idosos'] = df_polucao_final[['60 - 64 anos', '65 - 69 anos', '70 - 74 anos', '75 - 79 anos', 
                                                        '80 - 84 anos', '85 ou mais anos']].sum(axis=1)

    # Proporções de cada faixa etária
    df_polucao_final['Proporção Crianças'] = (df_polucao_final['Total Crianças'] / df_polucao_final['Total População']) * 100
    df_polucao_final['Proporção Adolescentes'] = (df_polucao_final['Total Adolescentes'] / df_polucao_final['Total População']) * 100
    df_polucao_final['Proporção Jovens Adultos'] = (df_polucao_final['Total Jovens Adultos'] / df_polucao_final['Total População']) * 100
    df_polucao_final['Proporção Adultos'] = (df_polucao_final['Total Adultos'] / df_polucao_final['Total População']) * 100
    df_polucao_final['Proporção Idosos'] = (df_polucao_final['Total Idosos'] / df_polucao_final['Total População']) * 100

    df_polucao_final = df_polucao_final.drop('Total_x', axis=1)

    return df_polucao_final.reset_index(drop=True)

# Função para listar cidades por grupo etário
def cidades_por_grupo(df, categoria='Todos', n=5):
    categorias = {
        'Crianças': 'Proporção Crianças',
        'Adolescentes': 'Proporção Adolescentes',
        'Jovens Adultos': 'Proporção Jovens Adultos',
        'Adultos': 'Proporção Adultos',
        'Idosos': 'Proporção Idosos',
        'Populosas': 'Total População'
    }
    
    if categoria not in categorias and categoria != 'Todos':
        print("Categoria inválida!")
        return
    
    if categoria == 'Todos':
        categorias_list = categorias.keys()
    else:
        categorias_list = [categoria]

    for cat in categorias_list:
        cidade_ordenada = df[['Ano', 'Região', categorias[cat]]].sort_values(by=categorias[cat], ascending=False).head(n)
        print(f"\nCidades com mais {cat} (%) - Top {n}:")
        for _, row in cidade_ordenada.iterrows():
            proporcao = row[categorias[cat]]
            print(f"{row['Região']}: {int(proporcao)}%")

def trata_df_populacao_densidade(df_densidade_populacional):

    df_densidade_populacional = df_densidade_populacional.dropna(axis=1, how='all')

    df_densidade_populacional = df_densidade_populacional[df_densidade_populacional['03. Nome Região (Portugal)'].notnull()]

    last_year = df_densidade_populacional['01. Ano'].max()
    df_densidade_populacional = df_densidade_populacional[df_densidade_populacional['01. Ano'] == last_year]

    df_densidade_populacional = df_densidade_populacional[df_densidade_populacional['03. Nome Região (Portugal)'] != 'Portugal']

    df_densidade_populacional = df_densidade_populacional[['01. Ano', '03. Nome Região (Portugal)', '04. Âmbito Geográfico', '10. Valor']]

    df_densidade_populacional.rename(columns={
        '01. Ano': 'Ano',
        '03. Nome Região (Portugal)': 'Região',
        '04. Âmbito Geográfico': 'Âmbito Geográfico',
        '10. Valor': 'Valor'
    }, inplace=True)

    df_densidade_populacional = df_densidade_populacional.astype({
        'Ano': 'int',
        'Região': 'str',
        'Âmbito Geográfico': 'str',
        'Valor': 'float'
    })

    return df_densidade_populacional.reset_index(drop=True)

def trata_crimes_catetoria(df_seguranca_crime_catetoria):
    
    df_seguranca_crime_catetoria = df_seguranca_crime_catetoria.dropna(axis=1, how='all')
    df_seguranca_crime_catetoria = df_seguranca_crime_catetoria[df_seguranca_crime_catetoria['02. Nome Região (Portugal)'] != 'Portugal']

    last_year = df_seguranca_crime_catetoria['01. Ano'].max()
    df_seguranca_crime_catetoria = df_seguranca_crime_catetoria[df_seguranca_crime_catetoria['01. Ano'] == last_year]

    df_seguranca_crime_catetoria = df_seguranca_crime_catetoria[['01. Ano', '02. Nome Região (Portugal)', '03. Âmbito Geográfico', '04. Filtro 1', '09. Valor']]

    df_seguranca_crime_catetoria.rename(columns={
        '01. Ano': 'Ano',
        '02. Nome Região (Portugal)': 'Região',
        '03. Âmbito Geográfico': 'Âmbito Geográfico',
        '04. Filtro 1': 'Categoria',
        '09. Valor': 'Valor'
    }, inplace=True)

    df_seguranca_crime_catetoria = df_seguranca_crime_catetoria.astype({
        'Ano': 'int',
        'Região': 'str',
        'Âmbito Geográfico': 'str',
        'Categoria': 'str',
        'Valor': 'float'
    })

    # filtering the dataframe to show only rows where 'Total' is not null
    df_seguranca_crime_catetoria = df_seguranca_crime_catetoria[df_seguranca_crime_catetoria['Valor'].notnull()]

    df_seg_categoria = df_seguranca_crime_catetoria.pivot_table(
            index=['Ano', 'Região'],
            columns='Categoria',
            values='Valor',
            aggfunc='sum',
            fill_value=0
    )

    return df_seg_categoria

def trata_crimes_tipo(df_seg_crimes_tipo):
    df_seg_crimes_tipo = df_seg_crimes_tipo.dropna(axis=1, how='all')
    df_seg_crimes_tipo = df_seg_crimes_tipo[df_seg_crimes_tipo['03. Nome Região (Portugal)'] != 'Portugal']
    df_seg_crimes_tipo = df_seg_crimes_tipo[df_seg_crimes_tipo['03. Nome Região (Portugal)'].notna()]

    last_year = df_seg_crimes_tipo['01. Ano'].max()
    df_seg_crimes_tipo = df_seg_crimes_tipo[df_seg_crimes_tipo['01. Ano'] == last_year]

    df_seg_crimes_tipo = df_seg_crimes_tipo.drop( columns= ['09. Símbolo','08. Escala'], axis=1)

    df_seg_crimes_tipo = df_seg_crimes_tipo[['01. Ano', '03. Nome Região (Portugal)', '04. Âmbito Geográfico', '05. Filtro 1', '10. Valor']]

    df_seg_crimes_tipo.rename(columns={
        '01. Ano': 'Ano',
        '03. Nome Região (Portugal)': 'Região',
        '04. Âmbito Geográfico': 'Âmbito Geográfico',
        '05. Filtro 1': 'Tipo de Crime',
        '10. Valor': 'Valor'
    }, inplace=True)

    df_seg_crimes_tipo = df_seg_crimes_tipo.astype({
        'Ano': 'int',
        'Região': 'str',
        'Âmbito Geográfico': 'str',
        'Tipo de Crime': 'str',
        'Valor': 'float'
    })

    df_seg_crimes_tipo = df_seg_crimes_tipo.pivot_table(
            index=['Ano', 'Região'],
            columns='Tipo de Crime',
            values='Valor',
            aggfunc='sum',
            fill_value=0
    )

    return df_seg_crimes_tipo.reset_index(drop=False)

def trata_hospitais(df_saude_hospitais):
    df_saude_hospitais = df_saude_hospitais.dropna(axis=1, how='all')
    df_saude_hospitais = df_saude_hospitais[df_saude_hospitais['02. Nome Região (Portugal)'] != 'Portugal']

    last_year = df_saude_hospitais['01. Ano'].max()
    df_saude_hospitais = df_saude_hospitais[df_saude_hospitais['01. Ano'] == last_year]

    df_saude_hospitais.drop(columns=['08. Símbolo'], inplace=True)

    df_saude_hospitais.rename(columns={
            '01. Ano': 'Ano',
            '02. Nome Região (Portugal)': 'Região',
            '03. Âmbito Geográfico': 'Âmbito Geográfico',
            '04. Filtro 1': 'Tipo hospital',
            '09. Valor': 'Valor'
        }, inplace=True)

    df_saude_hospitais = df_saude_hospitais.astype({
            'Ano': 'int',
            'Região': 'str',
            'Âmbito Geográfico': 'str',
            'Tipo hospital': 'str',
            'Valor': 'float'
        })

    df_saude_hospitais = df_saude_hospitais.pivot_table(
                index=['Ano', 'Região'],
                columns='Tipo hospital',
                values='Valor',
                aggfunc='sum',
                fill_value=0
        )

    df_saude_hospitais.reset_index(inplace=True)

    return df_saude_hospitais

def trata_farmacias(df_saude_farmacias):

    df_saude_farmacias = df_saude_farmacias.dropna(axis=1, how='all')
    df_saude_farmacias = df_saude_farmacias[df_saude_farmacias['02. Nome Região (Portugal)'] != 'Portugal']

    if df_saude_farmacias['08. Símbolo'].nunique() == 1:
        df_saude_farmacias = df_saude_farmacias.drop(columns=['08. Símbolo'])

    last_year = df_saude_farmacias['01. Ano'].max()
    df_saude_farmacias = df_saude_farmacias[df_saude_farmacias['01. Ano'] == last_year]

    df_saude_farmacias.rename(columns={
            '01. Ano': 'Ano',
            '02. Nome Região (Portugal)': 'Região',
            '03. Âmbito Geográfico': 'Âmbito Geográfico',
            '09. Valor': 'Valor'
        }, inplace=True)

    df_saude_farmacias = df_saude_farmacias.astype({
            'Ano': 'int',
            'Região': 'str',
            'Âmbito Geográfico': 'str',
            'Valor': 'float'
        })
    
    return df_saude_farmacias

def trata_ensino_n_superior(df_edu_ensino_n_superior):
    df_edu_ensino_n_superior = df_edu_ensino_n_superior.dropna(axis=1, how='all')
    df_edu_ensino_n_superior = df_edu_ensino_n_superior[df_edu_ensino_n_superior['02. Nome Região (Portugal)'] != 'Portugal']

    df_edu_ensino_n_superior = df_edu_ensino_n_superior.drop(columns=['08. Símbolo'])

    last_year = df_edu_ensino_n_superior['01. Ano'].max()
    df_edu_ensino_n_superior = df_edu_ensino_n_superior[df_edu_ensino_n_superior['01. Ano'] == last_year]

    df_edu_ensino_n_superior.rename(columns={
            '01. Ano': 'Ano',
            '02. Nome Região (Portugal)': 'Região',
            '03. Âmbito Geográfico': 'Âmbito Geográfico',
            '04. Filtro 1': 'Nível de Ensino',
            '05. Filtro 2': 'Tipo de Escola',
            '09. Valor': 'Valor'
        }, inplace=True)

    df_edu_ensino_n_superior = df_edu_ensino_n_superior.astype({
            'Ano': 'int',
            'Região': 'str',
            'Âmbito Geográfico': 'str',
            'Nível de Ensino': 'str',
            'Tipo de Escola': 'str',
            'Valor': 'float'
        })

    df_edu_escola_nivel = df_edu_ensino_n_superior.pivot_table(
            index=['Ano', 'Região'],
            columns='Nível de Ensino',
            values='Valor',
            aggfunc='sum',
            fill_value=0
        )

    df_edu_escola_nivel.reset_index(inplace=True)

    df_edu_escola_tipo= df_edu_ensino_n_superior.pivot_table(
            index=['Ano', 'Região'],
            columns='Tipo de Escola',
            values='Valor',
            aggfunc='sum',
            fill_value=0
        )

    df_edu_escola_tipo.reset_index(inplace=True)

    df_edu_escola_final = pd.merge(df_edu_escola_tipo, df_edu_escola_nivel, on=['Ano', 'Região'], how='left')
    df_edu_escola_final.reset_index(inplace=True)

    df_edu_escola_final.drop(columns=['index','Total_x'], inplace=True)
    df_edu_escola_final.rename(columns={'Total_y': 'Total Estabelecimentos'}, inplace=True)

    return df_edu_escola_final

In [ ]:
file_path = 'pordataFiles.json'
arquivos = read_json_file(file_path)

In [ ]:
dataframes = read_csv_files_from_folders(folderPathPorData)

In [ ]:
df_cultura_cinemas = find_df_by_name(dataframes, '623-recintos-de-cinema')
df_cultura_cinemas = trata_df_cultura(df_cultura_cinemas)
lista_cidades_cinemas(df_cultura_cinemas)

In [ ]:
df_cultura_espetaculos = find_df_by_name(dataframes, '631-sessoes-de-espetaculos-ao-vivo')
df_cultura_espetaculos = trata_df_cultura(df_cultura_espetaculos)
lista_cidades_cinemas(df_cultura_espetaculos)

In [ ]:
df_populaccao_residente = find_df_by_name(dataframes, '1-populacao-residente-por-sexo-e-grupo-etario')
df_populaccao_final = trata_df_populacao_residente(df_populaccao_residente)
cidades_por_grupo(df_populaccao_final, categoria='Todos', n = 10) # Todos Crianças Adolescentes Jovens Adultos Adultos Idosos Populosas

In [ ]:
df_densidade_populacional = find_df_by_name(dataframes, '4-densidade-populacional')
df_densidade_populacional = trata_df_populacao_densidade(df_densidade_populacional)

In [ ]:
df_densidade_populacional = df_densidade_populacional.sort_values(by=['Valor'], ascending=False).reset_index(drop=True)
print("Densidade populacional por região:")

In [97]:
df_seguranca_crime_catetoria = find_df_by_name(dataframes, '560-crimes-por-categoria')
df_seg_categoria = trata_crimes_catetoria(df_seguranca_crime_catetoria)

In [102]:
df_seg_crimes_tipo = find_df_by_name(dataframes, '559-crimes-registados-pelas-policias-por-tipo-de-crime')
df_seg_crimes_tipo = trata_crimes_tipo(df_seg_crimes_tipo)

In [144]:
df_saude_hospitais = find_df_by_name(dataframes, '470-hospitais-por-natureza-institucional')
df_saude_hospitais = trata_hospitais(df_saude_hospitais)
df_saude_hospitais

Tipo hospital,Ano,Região,Parceria público-privada,Privado,Público,Total
0,2023,Abrantes,0.0,0.0,1.0,1.0
1,2023,Aguiar da Beira,0.0,0.0,0.0,0.0
2,2023,Alandroal,0.0,0.0,0.0,0.0
3,2023,Albergaria-a-Velha,0.0,0.0,0.0,0.0
4,2023,Albufeira,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...
303,2023,Vouzela,0.0,0.0,0.0,0.0
304,2023,Águeda,0.0,0.0,1.0,1.0
305,2023,Évora,0.0,1.0,1.0,2.0
306,2023,Ílhavo,0.0,0.0,0.0,0.0


In [145]:
df_saude_farmacias = find_df_by_name(dataframes, '464-farmacias')
df_saude_farmacias = trata_farmacias(df_saude_farmacias)
df_saude_farmacias

,Ano,Região,Âmbito Geográfico,Valor
6502,2023,Abrantes,Município,15.0
6503,2023,Águeda,Município,14.0
6504,2023,Aguiar da Beira,Município,2.0
6505,2023,Alandroal,Município,4.0
6506,2023,Albergaria-a-Velha,Município,9.0
...,...,...,...,...
6805,2023,Vimioso,Município,3.0
6806,2023,Vinhais,Município,3.0
6807,2023,Viseu,Município,26.0
6808,2023,Vizela,Município,4.0


In [177]:
df_edu_ensino_n_superior =  find_df_by_name(dataframes, '379-estabelecimentos-de-ensino-nao-superior-por-0')
df_edu_escola_final = trata_ensino_n_superior(df_edu_ensino_n_superior)
df_edu_escola_final

,Ano,Região,Privado,Público,Escola artística,Escola básica,Escola básica e secundária,Escola profissional,Escola secundária,Jardim-de-infância,Total Estabelecimentos
0,2023,Abrantes,12.0,38.0,0.0,26.0,8.0,2.0,0.0,14.0,50.0
1,2023,Aguiar da Beira,0.0,14.0,0.0,8.0,2.0,0.0,0.0,4.0,14.0
2,2023,Alandroal,0.0,8.0,0.0,6.0,0.0,0.0,0.0,2.0,8.0
3,2023,Albergaria-a-Velha,20.0,44.0,0.0,30.0,2.0,2.0,2.0,28.0,64.0
4,2023,Albufeira,14.0,46.0,0.0,32.0,0.0,0.0,4.0,24.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...
303,2023,Vouzela,4.0,22.0,0.0,16.0,0.0,2.0,2.0,6.0,26.0
304,2023,Águeda,32.0,36.0,0.0,26.0,2.0,0.0,4.0,36.0,68.0
305,2023,Évora,40.0,76.0,0.0,50.0,2.0,2.0,6.0,56.0,116.0
306,2023,Ílhavo,18.0,50.0,0.0,42.0,0.0,0.0,4.0,22.0,68.0


In [ ]:
df_edu_ensino_n_superior = df_edu_ensino_n_superior.dropna(axis=1, how='all')
df_edu_ensino_n_superior = df_edu_ensino_n_superior[df_edu_ensino_n_superior['02. Nome Região (Portugal)'] != 'Portugal']

df_edu_ensino_n_superior = df_edu_ensino_n_superior.drop(columns=['08. Símbolo'])

last_year = df_edu_ensino_n_superior['01. Ano'].max()
df_edu_ensino_n_superior = df_edu_ensino_n_superior[df_edu_ensino_n_superior['01. Ano'] == last_year]

df_edu_ensino_n_superior.rename(columns={
        '01. Ano': 'Ano',
        '02. Nome Região (Portugal)': 'Região',
        '03. Âmbito Geográfico': 'Âmbito Geográfico',
        '04. Filtro 1': 'Nível de Ensino',
        '05. Filtro 2': 'Tipo de Escola',
        '09. Valor': 'Valor'
    }, inplace=True)

df_edu_ensino_n_superior = df_edu_ensino_n_superior.astype({
        'Ano': 'int',
        'Região': 'str',
        'Âmbito Geográfico': 'str',
        'Nível de Ensino': 'str',
        'Tipo de Escola': 'str',
        'Valor': 'float'
    })

df_edu_escola_nivel = df_edu_ensino_n_superior.pivot_table(
        index=['Ano', 'Região'],
        columns='Nível de Ensino',
        values='Valor',
        aggfunc='sum',
        fill_value=0
    )

df_edu_escola_nivel.reset_index(inplace=True)

df_edu_escola_tipo= df_edu_ensino_n_superior.pivot_table(
        index=['Ano', 'Região'],
        columns='Tipo de Escola',
        values='Valor',
        aggfunc='sum',
        fill_value=0
    )

df_edu_escola_tipo.reset_index(inplace=True)

df_edu_escola_final = pd.merge(df_edu_escola_tipo, df_edu_escola_nivel, on=['Ano', 'Região'], how='left')
df_edu_escola_final.reset_index(inplace=True)

df_edu_escola_final.drop(columns=['index','Total_x'], inplace=True)
df_edu_escola_final.rename(columns={'Total_y': 'Total Estabelecimentos'}, inplace=True)


In [173]:
df_edu_escola_final

,Ano,Região,Privado,Público,Escola artística,Escola básica,Escola básica e secundária,Escola profissional,Escola secundária,Jardim-de-infância,Total Estabelecimentos
0,2023,Abrantes,12.0,38.0,0.0,26.0,8.0,2.0,0.0,14.0,50.0
1,2023,Aguiar da Beira,0.0,14.0,0.0,8.0,2.0,0.0,0.0,4.0,14.0
2,2023,Alandroal,0.0,8.0,0.0,6.0,0.0,0.0,0.0,2.0,8.0
3,2023,Albergaria-a-Velha,20.0,44.0,0.0,30.0,2.0,2.0,2.0,28.0,64.0
4,2023,Albufeira,14.0,46.0,0.0,32.0,0.0,0.0,4.0,24.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...
303,2023,Vouzela,4.0,22.0,0.0,16.0,0.0,2.0,2.0,6.0,26.0
304,2023,Águeda,32.0,36.0,0.0,26.0,2.0,0.0,4.0,36.0,68.0
305,2023,Évora,40.0,76.0,0.0,50.0,2.0,2.0,6.0,56.0,116.0
306,2023,Ílhavo,18.0,50.0,0.0,42.0,0.0,0.0,4.0,22.0,68.0


,Ano,Região,Âmbito Geográfico,Nível de Ensino,Tipo de Escola,Valor
149268,2023,Abrantes,Município,Escola artística,Privado,NaN
149269,2023,Abrantes,Município,Escola artística,Público,NaN
149270,2023,Abrantes,Município,Escola artística,Total,NaN
149271,2023,Abrantes,Município,Escola básica,Privado,1.0
149272,2023,Abrantes,Município,Escola básica,Público,12.0
...,...,...,...,...,...,...
155731,2023,Vouzela,Município,Jardim-de-infância,Público,2.0
155732,2023,Vouzela,Município,Jardim-de-infância,Total,3.0
155733,2023,Vouzela,Município,Total,Privado,2.0
155734,2023,Vouzela,Município,Total,Público,11.0


In [104]:
# show dataframes values: df_folder, df_name
for df_info in dataframes:
    print(f"Pasta: {df_info['df_folder']} - DataFrame: {df_info['df_name']}")

Pasta: cultura - DataFrame: 623-recintos-de-cinema
Pasta: cultura - DataFrame: 631-sessoes-de-espetaculos-ao-vivo
Pasta: economia - DataFrame: 294-depositos-de-clientes-nos-bancos-caixas-economicas-e
Pasta: economia - DataFrame: 295-estabelecimentos-de-bancos-caixas-economicas-e-caixas
Pasta: educacao - DataFrame: 375-estabelecimentos-de-ensino-superior-por-subsistema
Pasta: educacao - DataFrame: 379-estabelecimentos-de-ensino-nao-superior-por-0
Pasta: emprego - DataFrame: 439-desemprego-registado-nos-centros-de-emprego-por-grup
Pasta: emprego - DataFrame: 445-populacao-empregada-por-conta-de-outrem-por-nivel-de
Pasta: emprego - DataFrame: 447-populacao-empregada-por-sexo-nivel-de-escolaridade-e
Pasta: emprego - DataFrame: 845-Pessoal-ao-servico-nas-empresas-por-ramo-de-atividade
Pasta: migracoes - DataFrame: 354-populacao-estrangeira-com-estatuto-legal-de-residente
Pasta: moradia - DataFrame: 978-edificios-concluidos-por-tipo-de-obra-e-fim-que-se
Pasta: moradia - DataFrame: 980-fogos-